# Visualization: Trading Session

In [1]:
import pandas as pd
import numpy as np

import altair as alt
import seaborn as sns

from trading_bot.utils import switch_k_backend_device

### 1. Define parameters and Load model

In [2]:
from trading_bot.agent import Agent

model_name = 'model_debug_11.00.keras'
test_stock = 'data/SPY_Test_data.csv'
window_size = 10
debug = True

agent = Agent(window_size, pretrained=True, model_name=model_name)

2024-07-18 23:21:59.440627: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-07-18 23:21:59.440656: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2024-07-18 23:21:59.440663: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2024-07-18 23:21:59.440687: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-07-18 23:21:59.440709: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


### 2. Load test data

In [3]:
# read csv into dataframe
df = pd.read_csv(test_stock)
# filter out the desired features
df = df[['Date', 'Adj Close']]
# rename feature column names
df = df.rename(columns={'Adj Close': 'actual', 'Date': 'date'})
# convert dates from object to DateTime type
dates = df['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['date'] = dates

df.head()

/var/folders/y8/kbsmbsyx18908prd50by3b440000gn/T/ipykernel_59465/308547104.py:9: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dates = pd.to_datetime(dates, infer_datetime_format=True)


,date,actual
0,2023-07-03,437.668182
1,2023-07-05,437.017242
2,2023-07-06,433.595154
3,2023-07-07,432.500427
4,2023-07-10,433.595154


### 3. Running Eval

In [4]:
import logging
import coloredlogs

from trading_bot.utils import show_eval_result, switch_k_backend_device, get_stock_data
from trading_bot.methods import evaluate_model

coloredlogs.install(level='DEBUG')
switch_k_backend_device()

test_data = get_stock_data(test_stock)
initial_offset = test_data[1] - test_data[0]

test_result, history = evaluate_model(agent, test_data, window_size, debug)
show_eval_result(model_name, test_result, initial_offset)

2024-07-18 23:22:08 Becketts-MBP root[59465] DEBUG Checking for MPS support
2024-07-18 23:22:08.293716: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-07-18 23:22:08 Becketts-MBP root[59465] DEBUG Buy at: $437.67
2024-07-18 23:22:08 Becketts-MBP root[59465] DEBUG Buy at: $437.02
2024-07-18 23:22:08 Becketts-MBP root[59465] DEBUG Buy at: $433.60


2024-07-18 23:22:08 Becketts-MBP tensorflow[59465] WARNING 5 out of the last 5 calls to <function tf_get_state at 0x3188a8700> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
2024-07-18 23:22:08 Becketts-MBP root[59465] DEBUG Buy at: $432.50


2024-07-18 23:22:08 Becketts-MBP tensorflow[59465] WARNING 6 out of the last 6 calls to <function tf_get_state at 0x3188a8700> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
2024-07-18 23:22:08 Becketts-MBP root[59465] DEBUG Buy at: $433.60
2024-07-18 23:22:08 Becketts-MBP root[59465] DEBUG Buy at: $436.36
2024-07-18 23:22:08 Becketts-MBP root[59465] DEBUG Buy at: $439.87
2024-07-18 23:22:08 Becketts-MBP root[59465] DEBUG Buy at: $443.36
2024-07-18 23:22:08 Becketts-MBP root[

### 4. Visualize

In [5]:
def visualize(df, history, title="trading session"):
    # add history to dataframe
    position = [history[0][0]] + [x[0] for x in history]
    actions = ['HOLD'] + [x[1] for x in history]
    df['position'] = position
    df['action'] = actions
    
    # specify y-axis scale for stock prices
    scale = alt.Scale(domain=(min(min(df['actual']), min(df['position'])) - 50, max(max(df['actual']), max(df['position'])) + 50), clamp=True)
    
    # plot a line chart for stock positions
    actual = alt.Chart(df).mark_line(
        color='green',
        opacity=0.5
    ).encode(
        x='date:T',
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale)
    ).interactive(
        bind_y=False
    )
    
    # plot the BUY and SELL actions as points
    points = alt.Chart(df).transform_filter(
        alt.datum.action != 'HOLD'
    ).mark_point(
        filled=True
    ).encode(
        x=alt.X('date:T', axis=alt.Axis(title='Date')),
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale),
        color='action'
    ).interactive(bind_y=False)

    # merge the two charts
    chart = alt.layer(actual, points, title=title).properties(height=300, width=1000)
    
    return chart

In [6]:
chart = visualize(df, history, title=test_stock)
chart

alt.LayerChart(...)